In [9]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [10]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797868,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797869,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797870,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797871,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [11]:
data = pd.read_csv('общие_данные_2010t+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]


protests = data[data['Event Text'].isin(protest_events)].copy()
protests_0910 = protests[protests['year'].isin([2009, 2010])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('oktmo')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['oktmo']).reset_index(drop=True)



protests.to_csv('протесты_2010t+.csv', index=False)
protests1.to_csv('протесты2009_2010t+.csv', index=False)
protests1

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text,fprotets_year
0,2009,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,163362.0,0.0,...,0.0,0.0,0.0,0.0,163490.0,416.00,20553.0,11238.0,Criticize or denounce,2009
1,2009,Башкортостан,Альшеевский,80602000,0.80,0.00,200.0,3100.0,44384.0,0.0,...,0.0,0.0,0.0,0.0,44300.0,416.00,10346.3,241142.0,Criticize or denounce,2009
2,2009,Краснодарский,Анапа,3703000,0.90,2.22,300.0,21300.0,139761.0,0.0,...,0.0,0.0,0.0,0.0,141105.0,366.00,14102.0,98186.0,Criticize or denounce,2009
3,2009,Тверская,Андреапольский,28602000,15.60,0.00,0.0,0.0,13970.0,0.0,...,0.0,0.0,0.0,0.0,13792.0,618.06,10502.7,305086.0,Criticize or denounce,2009
4,2009,Архангельская,Архангельск,11701000,485.50,0.00,2000.0,1300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,416.00,22976.9,29445.0,Criticize or denounce,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2010,Тверская,Тверь,28701000,80.50,0.00,700.0,500.0,410410.0,0.0,...,11.1,16.0,4473.0,12482.0,403594.0,424.13,20254.4,15222.0,Criticize or denounce,2010
105,2010,Краснодарский,Тимашевский,3653000,19.00,0.00,100.0,9400.0,107768.0,0.0,...,13.9,14.3,1475.0,26041.1,105846.0,365.00,17644.1,150643.0,Demonstrate or rally,2010
106,2010,Вологодская,Тотемский,19646000,47.50,0.00,50.0,840.0,0.0,0.0,...,12.7,17.9,296.0,16863.0,23956.0,524.50,15125.5,819598.0,Criticize or denounce,2010
107,2010,Карачаево-Черкесская,Усть-Джегутинский,91635000,19.00,0.00,0.0,3100.0,50039.0,0.0,...,16.6,10.2,853.0,3230.8,51510.0,294.60,11440.1,94295.0,Demonstrate or rally,2010


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [12]:
lol = pd.read_csv("протесты_2010t+.csv") 
sobytia = pd.read_csv("общие_данные_2010t+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623222,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623223,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623224,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623225,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [13]:
lol = pd.read_csv("протесты2009_2010t+.csv") 
sobytia = pd.read_csv("общие_данные_2010t+.csv")

df_2011 = sobytia[sobytia['year'] == 2008]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2008_2010t+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2008,Хакасия,Абакан,95701000,21.1,0.00,381.0,6300.0,0.0,0.0,...,-13052.0,0.0,0.0,0.0,0.0,0.0,411.00,18275.2,11238.0,"Arrest, detain, or charge with legal action"
1,2008,Ставропольский,Александровский,7602000,0.0,0.00,0.0,0.0,0.0,-3.6,...,6993.0,12.6,16.2,632.0,0.0,0.0,359.48,7463.3,0.0,NaN
2,2008,Ставропольский,Александровский,7602402,0.0,1.46,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7866.5,37450.0,NaN
3,2008,Краснодарский,Анапа,3703000,1.0,11.30,1600.0,34000.0,0.0,0.0,...,-41505.0,0.0,0.0,0.0,0.0,0.0,360.00,12470.0,98186.0,"Arrest, detain, or charge with legal action"
4,2008,Архангельская,Архангельск,11701000,474.7,0.00,3065.0,3181.0,0.0,0.0,...,-44553.0,0.0,0.0,0.0,0.0,0.0,0.00,21012.6,29445.0,Make an appeal or request
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2008,Дагестан,Хасавюрт,82735000,358.2,0.00,4600.0,4600.0,0.0,0.0,...,22137.0,0.0,0.0,0.0,0.0,0.0,0.00,8241.9,3848.0,Make statement
78,2008,Ростовская,Целинский,60656000,12.1,4.11,0.0,6500.0,0.0,0.0,...,9307.0,0.0,0.0,0.0,0.0,33.6,416.28,8707.9,212960.0,NaN
79,2008,Тульская,Чернский,70646000,0.0,0.00,0.0,14700.0,0.0,0.0,...,9451.0,0.0,0.0,0.0,0.0,0.0,533.64,8898.8,161429.0,Investigate
80,2008,Адыгея,Шовгеновский,79640000,0.0,0.00,0.0,0.0,0.0,0.0,...,2752.0,13.4,17.6,210.0,0.0,17186.0,312.80,6265.3,0.0,fight with small arms and light weapons


Создание регрессии

In [14]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2009_2010t+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t+.csv")
df_events_full = pd.read_csv("общие_данные_2010t+.csv")

required_years = {2008.0, 2011.0, 2012.0, 2013.0}

df_events = df_events[df_events['year'].isin([2009, 2010])].reset_index(drop=True)

features = [
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',
            # 'Квартиры',  
            
            # 'Население', 
            # 'Прибыль',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)


exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     "Александровский": ["Петровский", "Благодарненский", "Новоселицкий",
                        "Георгиевский", "Минераловодский", "Андроповский", "Грачёвский"],
    "Петропавловск-Камчатский": ["Вилючинский", "Елизовский"],
    "Советский": ["Новоселицкий", "Будённовский", "Степновский", "Кировский", "Георгиевский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"]
}
     

excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Ингушетия нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,...,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2009,2.79,0.00,400.0,860.0,0.00,0.00,...,5.60,0.0,0.0,139.0,0.00,0.00,16612.0,0.00,8861.0,0.178501
1,Хакасия,Таштыпский,95625000,2010,10830.00,0.00,0.0,10.0,600.00,26.10,...,34717.00,0.0,0.0,1600.0,624.00,11.03,1174.0,0.87,661055.0,1.633220
2,Башкортостан,Альшеевский,80602000,2009,0.80,0.00,200.0,3100.0,723.00,0.00,...,5.20,0.0,0.0,1000.0,699.00,0.00,29082.0,0.00,245588.0,0.140861
3,Башкортостан,Аскинский,80604000,2009,5.10,0.00,0.0,2600.0,1666.00,0.00,...,0.60,0.0,200.0,1600.0,1944.00,0.00,-3479.0,8.50,428891.0,0.880472
4,Башкортостан,Учалинский,80653000,2009,3.50,0.00,0.0,12700.0,1017.10,32.70,...,10.40,0.0,0.0,0.0,1168.60,25.20,6675.0,0.00,323221.0,0.472496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Новосибирская,Новосибирский,50640000,2010,25.90,0.00,600.0,16740.0,0.00,0.00,...,2.85,0.0,0.0,1055.0,0.00,0.00,18258.0,0.65,251771.0,0.293736
93,Омская,Омск,52701000,2010,230.00,2.82,2300.0,15100.0,725.69,19.61,...,54.20,0.0,300.0,1100.0,607.70,5.75,-1430.0,0.00,165195.0,0.609994
94,Камчатский,Петропавловск-Камчатский,30701000,2010,82.10,0.00,0.0,0.0,1469.77,47.80,...,8.90,0.0,0.0,0.0,1200.00,5.03,-21725.0,0.00,3510.0,2.495641
95,Тамбовская,Тамбов,68701000,2010,318.10,0.00,100.0,1000.0,946.50,42.80,...,45.70,0.0,120.0,100.0,615.02,28.26,-38032.0,0.00,7778.0,1.113532


In [15]:
df_2012 = pd.read_csv('прототип_данных_2010t+.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year',

    'ПотреблениеЭ_event', 
    'ПотреблениеВ_event', 
    'Бюджет_event',
    'БезДорог_event', 
    'Земля_event',
   

    'ПотреблениеЭ_protest', 
    'ПотреблениеВ_protest', 
    'Бюджет_protest',
    'БезДорог_protest', 
    'Земля_protest',
  
]]


from2012.to_csv('связка_махаланобис_2010t+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,...,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,ПотреблениеЭ_protest,ПотреблениеВ_protest,Бюджет_protest,БезДорог_protest,Земля_protest
0,Абакан,95701000,2.79,0.00,400.0,860.0,Саяногорск,95708000,5.6,0.0,...,0.00,0.00,16612.0,0.00,8861.0,0.00,0.00,37155.0,0.0,11238.0
1,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Орджоникидзевский,95620000,34717.0,0.0,...,624.00,11.03,1174.0,0.87,661055.0,600.00,26.10,2401.0,2.0,1998988.0
2,Учалинский,80653000,3.50,0.00,0.0,12700.0,Мелеузовский,80641000,10.4,0.0,...,1168.60,25.20,6675.0,0.00,323221.0,1017.10,32.70,-5294.0,0.3,455300.0
3,Благовещенск,80615101,0.00,0.00,0.0,3500.0,Старотуймазинский,80651455,0.0,0.0,...,0.00,0.00,-940.0,0.00,6000.0,0.00,0.00,-968.0,0.0,6466.0
4,Анапа,3703000,0.90,2.22,300.0,21300.0,Павловский,3639000,18.4,0.0,...,136.90,18.35,-59963.0,0.00,178880.0,179.00,36.00,-71274.0,0.0,98186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Междуреченск,87640153,0.00,0.00,0.0,318.0,Брыкаланск,87604410,0.0,0.0,...,0.00,0.00,353.0,0.00,221.0,0.00,0.00,375.0,0.0,335.0
67,Омск,52701000,230.00,2.82,2300.0,15100.0,Марьяновский,52630000,54.2,0.0,...,607.70,5.75,-1430.0,0.00,165195.0,725.69,19.61,-50359.0,0.0,56686.0
68,Петропавловск-Камчатский,30701000,82.10,0.00,0.0,0.0,Палана,30851000,8.9,0.0,...,1200.00,5.03,-21725.0,0.00,3510.0,1469.77,47.80,-275221.0,0.0,36214.0
69,Тамбов,68701000,318.10,0.00,100.0,1000.0,Мичуринск,68715000,45.7,0.0,...,615.02,28.26,-38032.0,0.00,7778.0,946.50,42.80,-124957.0,0.0,9089.0


In [16]:
df = pd.read_csv('общие_данные_2010t+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',
        'region_name',
        'ПотреблениеЭ', 
        'ПотреблениеВ', 
        'Бюджет',
        'БезДорог', 
        'Земля',
      ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2009, 2010]),
               'fprotets_year'] = pd.NA
    
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,...,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,fprotets_year
0,2008,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,0.00,0.00,...,0.00,0.0,132.0,Хакасия,0.00,0.00,-44395.0,0.0,8861.0,NaN
1,2009,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,0.00,0.00,...,0.00,0.0,139.0,Хакасия,0.00,0.00,16612.0,0.0,8861.0,2009.0
2,2010,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,1306.43,41.90,...,0.00,0.0,416.0,Хакасия,1150.72,35.17,-11411.0,0.0,11533.5,2009.0
3,2011,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,1304.43,41.90,...,0.00,0.0,0.0,Хакасия,1037.15,34.11,-911.0,0.0,11533.5,NaN
4,2012,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,1109.80,40.70,...,0.00,0.0,496.9,Хакасия,1153.76,24.28,203616.0,0.0,11533.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,2016,Усть-Джегутинский,91635000,0.00,27.56,0.0,7935.0,Карачаево-Черкесская,706.00,7.00,...,16.62,310.0,2500.0,Карачаево-Черкесская,0.00,0.00,-1276.4,0.0,53494.0,NaN
913,2017,Усть-Джегутинский,91635000,0.00,38.92,0.0,5450.0,Карачаево-Черкесская,706.00,7.00,...,16.62,310.0,2500.0,Карачаево-Черкесская,0.00,0.00,1989.0,0.0,53494.0,NaN
914,2018,Усть-Джегутинский,91635000,0.00,38.92,0.0,250.0,Карачаево-Черкесская,707.10,7.60,...,16.62,0.0,1485.0,Карачаево-Черкесская,0.00,0.00,-4205.5,0.0,53494.0,NaN
915,2019,Усть-Джегутинский,91635000,0.00,37.70,0.0,0.0,Карачаево-Черкесская,707.10,7.59,...,16.62,0.0,565.0,Карачаево-Черкесская,0.00,0.00,1825.9,0.0,53494.0,NaN
